## Import deps

In [1]:
!pip install mlable tokun

%load_ext tensorboard

In [14]:
import datetime
import functools
import itertools
import math
import os
import random
import urllib.request

import tensorflow as tf
import tensorflow_datasets as tfds

import mlable.data
import mlable.io
import mlable.layers.reshaping
import mlable.layers.transformer
import mlable.optimizers

import tokun.data
import tokun.evaluation
import tokun.meta
import tokun.model
import tokun.pipeline

In [3]:
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.15.0


## Setup the GPU / TPU

In [4]:
tf.debugging.set_log_device_placement(False)

CPU = tf.config.list_logical_devices('CPU')
GPU = tf.config.list_logical_devices('GPU')
TPU = tf.config.list_logical_devices('TPU')

if TPU:
    RESOLVER = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(RESOLVER)
    tf.tpu.experimental.initialize_tpu_system(RESOLVER)
    DISTRIBUTION_STRATEGY = tf.distribute.TPUStrategy(RESOLVER)
elif GPU:
    DISTRIBUTION_STRATEGY = tf.distribute.MirroredStrategy(GPU)
else:
    DISTRIBUTION_STRATEGY = tf.distribute.MirroredStrategy(CPU)

print(DISTRIBUTION_STRATEGY)

## Mode

In [5]:
# TOGGLE ######################################################################

IMPORT = False
TRAINING = True
RANDOM = True

## Defining The Metadata

In [7]:
# PARAMETERS ##################################################################

N_SEQUENCE_AXIS = 1
N_FEATURE_AXIS = -1

N_TOKEN_DIM = [4, 4] # G, for each block
N_ENCODING_DIM = 256 # U
N_EMBEDDING_DIM = N_ENCODING_DIM # E
N_HIDDEN_DIM = 4 * N_EMBEDDING_DIM # H
N_LATENT_DIM = N_EMBEDDING_DIM # L

N_EPOCHS = 16
N_EPOCHS_RAMPUP = 0
N_EPOCHS_SUSTAIN = 0

N_BATCH_DIM = 128 # number of samples per batch
N_SAMPLE_DIM = 256 # number of characters per sample (=> 4 * N_SAMPLE_DIM integers per sample)

R_MIN, R_MAX, R_EXP = tokun.meta.rates(pretrained=IMPORT, normalization=True, base=0.001)

In [8]:
# DERIVED #####################################################################

N_TOKEN_SIZES = list(itertools.accumulate(N_TOKEN_DIM, lambda x, y: x * y)) # in bytes
N_OFFSET_TICKS = [2 ** __i for __i in range(int(math.log(N_TOKEN_SIZES[-1] // 4, 2)))] # in characters

VERSION = tokun.meta.version(units=N_TOKEN_DIM, axis=N_SEQUENCE_AXIS)
DATETIME = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [9]:
# IMPORT ######################################################################

LABEL = '3.1'
URL_IMPORT = 'https://github.com/apehex/tokun/raw/main/models/{}/{}/{}.keras'.format(*VERSION, LABEL)
PATH_IMPORT = 'model.keras'

if IMPORT:
    urllib.request.urlretrieve(URL_IMPORT, PATH_IMPORT)

In [10]:
# EXPORT ######################################################################

PATH_LOG = os.path.join('.logs/', *VERSION, DATETIME)
PATH_EXPORT = 'model.keras'

## Loading The Data

In [11]:
# MLQA DATASET ################################################################

LANG = ['ar', 'de', 'en', 'es', 'hi', 'vi', 'zh']
MLQA_TRAIN = {__l: tfds.load('mlqa/' + __l, split='test', as_supervised=False, shuffle_files=True, data_dir='~/.cache/tensorflow/', batch_size=None) for __l in LANG}
MLQA_TEST = {__l: tfds.load('mlqa/' + __l, split='validation', as_supervised=False, shuffle_files=True, data_dir='~/.cache/tensorflow/', batch_size=None) for __l in LANG}

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating test examples...:   0%|          | 0/5335 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/5137 [00:00<?, ? examples/s]

Shuffling /root/.cache/tensorflow/mlqa/zh/incomplete.84AR1E_1.0.0/mlqa-test.tfrecord*...:   0%|          | 0/5…

Generating validation examples...:   0%|          | 0/504 [00:00<?, ? examples/s]

Shuffling /root/.cache/tensorflow/mlqa/zh/incomplete.84AR1E_1.0.0/mlqa-validation.tfrecord*...:   0%|         …

Dataset mlqa downloaded and prepared to /root/.cache/tensorflow/mlqa/zh/1.0.0. Subsequent calls will reuse this data.


In [12]:
# RANDOM DATASET ##############################################################

RANDOM_TRAIN = tokun.data.random_dataset(size=N_BATCH_DIM * 2**10, sample_size=N_SAMPLE_DIM, lower_plane=0, upper_plane=0x40000)
RANDOM_TEST = tokun.data.random_dataset(size=N_BATCH_DIM * 2**6, sample_size=N_SAMPLE_DIM, lower_plane=0, upper_plane=0x40000)

## Init The Model

In [15]:
# INIT ########################################################################

with DISTRIBUTION_STRATEGY.scope():
    if IMPORT and os.path.isfile(PATH_IMPORT):
        MODEL = tf.keras.models.load_model(PATH_IMPORT)
    else:
        MODEL = tokun.model.AutoEncoder(sequence_axis=N_SEQUENCE_AXIS, feature_axis=N_FEATURE_AXIS, token_dim=N_TOKEN_DIM, encoding_dim=N_ENCODING_DIM, embedding_dim=N_EMBEDDING_DIM, hidden_dim=N_HIDDEN_DIM, latent_dim=N_LATENT_DIM, activation='gelu')
    MODEL.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=R_MAX),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0., axis=-1, reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE, name='loss'),
        metrics=['accuracy'])


## Preprocess

In [16]:
# MLQA ########################################################################

PIPELINE = [
    # offset by 1 to 15 character => (1,) scalar bytes
    *[(functools.partial(tokun.pipeline.offset, ticks=__t), False) for __t in N_OFFSET_TICKS], # (offsets 0, ..., (2 ^ i) - 1) + (offsets 2 ^ i, ..., 2 ^ (i+1) - 1)
    # encode => (4 * S,) int (4 UTF-32 bytes per character)
    (functools.partial(tokun.pipeline.encode, token_size=N_TOKEN_SIZES[-1], sample_size=N_SAMPLE_DIM), True),
    # reshape => (4 * S,) int
    (functools.partial(tf.reshape, shape=(4 * N_SAMPLE_DIM,)), True),
    # one-hot encoding => (4 * S, E) int (bool)
    (functools.partial(tf.one_hot, depth=N_ENCODING_DIM, axis=-1), True),
    # replace sample inputs with (input, target) for supervised learning
    ((lambda x: (x, x)), True)]

OPERATIONS, REPLACE = zip(*PIPELINE)

MLQA_TRAIN = {__l: mlable.data.process(dataset=__d, feature='context', pipeline=OPERATIONS, replace=REPLACE) for __l, __d in MLQA_TRAIN.items()}
MLQA_TEST = {__l: mlable.data.process(dataset=__d, feature='context', pipeline=OPERATIONS, replace=REPLACE) for __l, __d in MLQA_TEST.items()}

In [17]:
# RANDOM ######################################################################

PIPELINE = [
    # reshape each sample => (G * S,) int
    (functools.partial(tf.reshape, shape=(4 * N_SAMPLE_DIM,)), True),
    # one-hot encoding => (G * S, E) int (bool)
    (functools.partial(tf.one_hot, depth=N_ENCODING_DIM, axis=-1), True),
    # replace sample inputs with (input, target) for supervised learning
    ((lambda x: (x, x)), True)]

OPERATIONS, REPLACE = zip(*PIPELINE)

RANDOM_TRAIN = mlable.data.process(dataset=RANDOM_TRAIN, feature='', pipeline=OPERATIONS, replace=REPLACE)
RANDOM_TEST = mlable.data.process(dataset=RANDOM_TEST, feature='', pipeline=OPERATIONS, replace=REPLACE)

In [18]:
# TOGGLE ######################################################################

DATASET_TRAIN = RANDOM_TRAIN if RANDOM else MLQA_TRAIN['ar'].concatenate(MLQA_TRAIN['en']).concatenate(MLQA_TRAIN['es']).concatenate(MLQA_TRAIN['de']).concatenate(MLQA_TRAIN['hi']).concatenate(MLQA_TRAIN['vi']).concatenate(MLQA_TRAIN['zh'])
DATASET_TEST = MLQA_TEST['ar'].concatenate(MLQA_TEST['en']).concatenate(MLQA_TEST['es']).concatenate(MLQA_TEST['de']).concatenate(MLQA_TEST['hi']).concatenate(MLQA_TEST['vi']).concatenate(MLQA_TEST['zh'])

In [19]:
# INSPECT #####################################################################

print(DATASET_TRAIN.element_spec)
print(DATASET_TEST.element_spec)

(TensorSpec(shape=(1024, 256), dtype=tf.float32, name=None), TensorSpec(shape=(1024, 256), dtype=tf.float32, name=None))
(TensorSpec(shape=(1024, 256), dtype=tf.float32, name=None), TensorSpec(shape=(1024, 256), dtype=tf.float32, name=None))


## Train

In [ ]:
# TRAIN #######################################################################

lr_callback = tf.keras.callbacks.LearningRateScheduler(functools.partial(mlable.optimizers.learning_rate_hokusai, lr_min=R_MIN, lr_max=R_MAX, lr_exp=R_EXP, rampup=N_EPOCHS_RAMPUP, sustain=N_EPOCHS_SUSTAIN), verbose=True)
cp_callback = tf.keras.callbacks.ModelCheckpoint(PATH_EXPORT, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=PATH_LOG)

if TRAINING:
    with DISTRIBUTION_STRATEGY.scope():
        TRAINING_HISTORY = MODEL.fit(
            x=DATASET_TRAIN.batch(N_BATCH_DIM).prefetch(tf.data.AUTOTUNE),
            batch_size=None,
            epochs=N_EPOCHS,
            validation_split=None,
            validation_data=DATASET_TEST.batch(N_BATCH_DIM).prefetch(tf.data.AUTOTUNE),
            validation_freq=list(range(1, N_EPOCHS + 1, 1)),
            verbose=2,
            callbacks=[lr_callback, cp_callback, tb_callback])


Epoch 1: LearningRateScheduler setting learning rate to 0.0001.
Epoch 1/8

Epoch 1: val_loss improved from inf to 0.03787, saving model to model.keras
1024/1024 - 304s - loss: 0.8168 - accuracy: 0.8681 - val_loss: 0.0379 - val_accuracy: 1.0000 - lr: 1.0000e-04 - 304s/epoch - 297ms/step

Epoch 2: LearningRateScheduler setting learning rate to 8.020000000000001e-05.
Epoch 2/8

Epoch 2: val_loss improved from 0.03787 to 0.01136, saving model to model.keras
1024/1024 - 289s - loss: 0.0060 - accuracy: 1.0000 - val_loss: 0.0114 - val_accuracy: 1.0000 - lr: 8.0200e-05 - 289s/epoch - 282ms/step

Epoch 3: LearningRateScheduler setting learning rate to 6.436000000000001e-05.
Epoch 3/8

Epoch 3: val_loss improved from 0.01136 to 0.00587, saving model to model.keras
1024/1024 - 290s - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0059 - val_accuracy: 1.0000 - lr: 6.4360e-05 - 290s/epoch - 283ms/step

Epoch 4: LearningRateScheduler setting learning rate to 5.168800000000001e-05.
Epoch 4/8

Epoch 4

In [ ]:
MODEL.summary()

## Export

In [ ]:
# SAMPLES #####################################################################

IO = {}
TOKENS = {__i: {} for __i in N_TOKEN_SIZES} # length in bytes
EMBEDDINGS = {__i: {} for __i in N_TOKEN_SIZES} # same

for __lang, __dataset in MLQA_TEST.items():
    # compute predictions
    __batch = iter(__dataset.batch(N_BATCH_DIM)) # iterate over batches of samples
    __input = next(__batch)[0] # take input only
    __output = MODEL(__input)
    # sample predictions (inputs, outputs)
    IO[__lang] = (__input, __output)

In [ ]:
# RANDOM ######################################################################

# predictions
__batch = iter(RANDOM_TEST.batch(N_BATCH_DIM))
__input = next(__batch)[0]
__output = MODEL(__input)
# sample predictions (inputs, outputs)
IO['rd'] = (__input, __output)

In [ ]:
# TOKENS ######################################################################

# unique (G ^ i)-tokens
for __lang, __sample in IO.items():
    for __size in TOKENS:
        TOKENS[__size][__lang] = tokun.pipeline.chunk(seq=tokun.pipeline.postprocess(__sample[0]), size=__size // 4, repeats=False)

# unique tokens, for all languages
for __size in TOKENS:
    TOKENS[__size]['all'] = list(set(__t for _, __s in TOKENS[__size].items() for __t in __s))

In [ ]:
# EMBEDDINGS ##################################################################

for __depth, __size in enumerate(N_TOKEN_SIZES):
    for __lang, __tokens in TOKENS[__size].items():
        # re-encode without token repeats
        __input = tokun.pipeline.preprocess(text=''.join(__tokens), groups=N_TOKEN_DIM, expand=N_SEQUENCE_AXIS * [1], flatten=True)
        # UTF-32 embedding
        __embedding = MODEL._encoder._encoder.layers[0](__input)
        # iterative CNN tokenization
        for __i in range(__depth + 1):
            __embedding = MODEL._encoder._encoder.layers[__i + 1](__embedding)
        # remove the (tokenized) padding
        EMBEDDINGS[__size][__lang] = tf.squeeze(__embedding)[:len(__tokens)] # TODO squeeze?

In [ ]:
# NEIGHBORHOODS ###############################################################

__unit = N_TOKEN_SIZES[-1]
__count = 256

TOKENS['local'] = {'all': []}
EMBEDDINGS['local'] = {'all': []}

for __lang, __tokens in TOKENS[__unit].items():
    # stats on the embeddings for the current language
    __std = tf.math.reduce_std(EMBEDDINGS[__unit][__lang], axis=0, keepdims=True)
    __radius = 2. ** (3 - math.log(__unit, 4)) * tf.reduce_mean(__std).numpy()
    # choose a single token
    __t = tokun.pipeline.preprocess(text=random.choice(__tokens), groups=N_TOKEN_DIM, expand=N_SEQUENCE_AXIS * [1], flatten=True)
    # encode it
    __e = MODEL._encoder(__t)
    # add noise to generate random neighbors
    __n = tokun.evaluation.neighbors(point=__e, radius=__radius, count=__count)
    # decode the noisy embeddings
    __d = MODEL._decoder(__n)
    # postprocess
    __m = tokun.pipeline.chunk(seq=tokun.pipeline.postprocess(__d), size=__unit // 4, repeats=True)
    # save
    TOKENS['local']['all'].extend(__m)
    EMBEDDINGS['local']['all'].append(tf.squeeze(__n))

# merge all the embedding tensors
EMBEDDINGS['local']['all'] = tf.concat(values=EMBEDDINGS['local']['all'], axis=0)

In [ ]:
# SAVE ########################################################################

for __size in TOKENS:
    mlable.io.write(data=[__c + ' ' + mlable.io.label(__c) for __c in TOKENS[__size]['all']][:8192], path='./metadata.' + str(__size) + '.label.tsv', tsv=False)
    mlable.io.write(data=TOKENS[__size]['all'][:8192], path='./metadata.' + str(__size) + '.tsv', tsv=False)
    mlable.io.write(data=EMBEDDINGS[__size]['all'].numpy()[:8192], path='./embeddings.' + str(__size) + '.tsv', tsv=True)

## Dataviz

In [ ]:
# DATA ########################################################################

SAMPLES = [
    """위키백과, 우리 모두의 백과사전.\nt-분포 확률적 임베딩(t-SNE)은 데이터의 차원 축소에 사용되는 기계 학습 알고리즘 중 하나로, 2002년 샘 로이스Sam Rowise와 제프리 힌튼에 의해 개발되었다.[1] t-SNE는 비선형 차원 축소 기법으로, 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용하게 사용된다. 구체적으로 t-SNE는 비슷한 데이터는 근접한 2, 3차원의 지점으로, 다른 데이터는 멀리 떨어진 지점으로 맵핑한다.""",
    """class Encoder(tf.keras.models.Model):\n    def __init__(self, depth: int, token_dim: int, encoding_dim: int, embedding_dim: int, latent_dim: int, batch_dim: int=None, attention: bool=False, **kwargs) -> None:\n        super(Encoder, self).__init__(**kwargs)\n        self._encoder = tf.keras.Sequential([\n            tf.keras.Input(shape=(encoding_dim,), batch_size=batch_dim, name='input'), # (B * G ^ D, U)\n            tf.keras.layers.Dense(units=embedding_dim, activation=None, use_bias=False, kernel_initializer='glorot_uniform', bias_initializer=None, name='embed-1'),] # (B * G ^ D, U) => (B * G ^ D, E)\n            + [tokun.layers.TokenizeBlock(left_axis=-2, right_axis=-1, token_dim=token_dim, latent_dim=latent_dim, attention=attention, name='tokenize' + (__i + 1) * '-4') for __i in range(depth)]) # (B * G ^ i, E) => (B * G ^ (i-1), E)\n\n    def call(self, x: tf.Tensor) -> tf.Tensor:\n        return self._encoder(x)\n""",
    """class AutoEncoder(tf.keras.models.Model):\n    def __init__(self, token_dim: int, encoding_dim: int, embedding_dim: int, latent_dim: int, batch_dim: int=None, **kwargs) -> None:\n        super(AutoEncoder, self).__init__(**kwargs)\n        self._encoder = Encoder(token_dim=token_dim, encoding_dim=encoding_dim, embedding_dim=embedding_dim, latent_dim=latent_dim, batch_dim=batch_dim)\n        self._decoder = Decoder(token_dim=token_dim, encoding_dim=encoding_dim, embedding_dim=embedding_dim, latent_dim=latent_dim, batch_dim=batch_dim)\n\n    def call(self, x: tf.Tensor) -> tf.Tensor:\n        return self._decoder(self._encoder(x))""",
    """class AutoEncoder(tf.keras.models.Model):\n  def __init__(self, token_dim: int, encoding_dim: int, embedding_dim: int, latent_dim: int, batch_dim: int=None, **kwargs) -> None:\n    super(AutoEncoder, self).__init__(**kwargs)\n    self._encoder = Encoder(token_dim=token_dim, encoding_dim=encoding_dim, embedding_dim=embedding_dim, latent_dim=latent_dim, batch_dim=batch_dim)\n    self._decoder = Decoder(token_dim=token_dim, encoding_dim=encoding_dim, embedding_dim=embedding_dim, latent_dim=latent_dim, batch_dim=batch_dim)\n\n  def call(self, x: tf.Tensor) -> tf.Tensor:\n    return self._decoder(self._encoder(x))"""]

In [ ]:
# COMPUTE ######################################################################

__i = 0
__x, __e, __p, __y = tokun.pipeline.sample(model=MODEL, text=SAMPLES[__i], groups=N_TOKEN_DIM, expand=N_SEQUENCE_AXIS * [1], flatten=True)

In [ ]:
print('# INPUT ################################################################\n\n' + SAMPLES[__i])
print('\n# OUTPUT ###############################################################\n\n' + __y)
print('\n# SCORE ################################################################\n\n' + str(tokun.evaluation.compare(SAMPLES[__i], __y)))

In [ ]:
# FROM DATASET ################################################################

# compute
__l = tokun.pipeline.postprocess(IO['de'][0])
__r = tokun.pipeline.postprocess(IO['de'][1])

# print
print(__l)
print(__r)
print(tokun.evaluation.compare(__l, __r))

In [ ]:
%tensorboard --logdir .logs

In [ ]:
print(tf.math.reduce_mean(EMBEDDINGS[N_TOKEN_SIZES[-1]]['en'], axis=0))
print(tf.math.reduce_std(EMBEDDINGS[N_TOKEN_SIZES[-1]]['en'], axis=0))

In [ ]:
__std = tf.math.reduce_std(EMBEDDINGS[N_TOKEN_SIZES[-1]]['en'], axis=0)
__noise = tf.random.normal(shape=(256,), mean=0., stddev=tf.math.reduce_mean(__std).numpy())

__x = tokun.pipeline.preprocess('tokun to can tokens', groups=N_TOKEN_DIM, expand=N_SEQUENCE_AXIS * [1], flatten=True)
__e = MODEL._encoder(__x)

print(tokun.pipeline.postprocess(MODEL._decoder(__e)))
print(tokun.pipeline.postprocess(MODEL._decoder(__e + 0.4 * __std)))
print(tokun.pipeline.postprocess(MODEL._decoder(__e + 0.2 * __noise)))

In [ ]:
__x = IO['en'][0]
MODEL._encoder(__x)